In [101]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StructType, DoubleType, IntegerType

In [102]:
spark = SparkSession.builder \
    .appName('cloud') \
    .master('local[*]') \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.30") \
    .getOrCreate()
    
spark

In [103]:
schema = StructType([
    StructField('entries', ArrayType(StructType([
        StructField('title', StringType(), True),
        StructField('price', StringType(), True),         
        StructField('sale-price', StringType(), True),   
        StructField('currency', StringType(), True),
        StructField('category-id', IntegerType(), True),   
        StructField('slug', StringType(), True),
        StructField('options', StringType(), True),
        StructField('link-shopee', StringType(), True),   
        StructField('link-zalo', StringType(), True),
        StructField('link-instagram', StringType(), True),
        StructField('description', StringType(), True),
        StructField('detail', StringType(), True),
        StructField('img', StringType(), True),
        StructField('sub-img', ArrayType(StringType()), True), 
        StructField('categorical_products', ArrayType(StructType([
            StructField('title', StringType(), True),
            StructField('price', StringType(), True),     
            StructField('sale-price', StringType(), True),
            StructField('currency', StringType(), True),
            StructField('options', StringType(), True),
            StructField('img', StringType(), True),
            StructField('sub-img', ArrayType(StringType()), True), 
        ])))
    ])))
])

In [104]:
df = spark.read.schema(schema).option('multiline', True).json('C:/Users/Admin/hnv-demo/src/data/category-products.json')

In [6]:
df.printSchema()

root
 |-- entries: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- sale-price: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- category-id: integer (nullable = true)
 |    |    |-- slug: string (nullable = true)
 |    |    |-- options: string (nullable = true)
 |    |    |-- link-shopee: string (nullable = true)
 |    |    |-- link-zalo: string (nullable = true)
 |    |    |-- link-instagram: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- detail: string (nullable = true)
 |    |    |-- img: string (nullable = true)
 |    |    |-- sub-img: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- categorical_products: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- title: s

In [7]:
df.show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [105]:
df_exploded = df.withColumn('entries_exploded', explode('entries'))

In [9]:
df_exploded.select('entries_exploded.*').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------+-------+----------+--------+-----------+-------------------------------------------------------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+--------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------

In [ ]:
df_exploded.select('entries_exploded').printSchema()

root
 |-- entries_exploded: struct (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- sale-price: string (nullable = true)
 |    |-- currency: string (nullable = true)
 |    |-- category-id: integer (nullable = true)
 |    |-- slug: string (nullable = true)
 |    |-- options: string (nullable = true)
 |    |-- link-shopee: string (nullable = true)
 |    |-- link-zalo: string (nullable = true)
 |    |-- link-instagram: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- detail: string (nullable = true)
 |    |-- img: string (nullable = true)
 |    |-- sub-img: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- categorical_products: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- price: string (nullable = true)
 |    |    |    |-- sale-price: string (nullable = t

In [106]:
products_df = df_exploded.withColumnRenamed('entries_exploded', 'e') \
    .select(
        'e.title', 
        'e.slug',
        expr('CAST (e.price AS DOUBLE) * 1000').alias('price'),
        col('e.sale-price').alias('sale_price'),
        col('e.category-id').alias('category_id'),
        'e.description',
        'e.detail',
        col('e.link-shopee').alias('link_shopee'),
        col('e.link-zalo').alias('link_zalo'),
        col('e.link-instagram').alias('link_instagram'),
        lit("").alias('meta_title'),
        lit("").alias("meta_description")
    ) \
    .withColumn('sale_price', expr('CAST (sale_price AS DOUBLE) * 1000')) \
    .withColumn('category_id', expr("CASE WHEN title LIKE 'Bình%' THEN 1 WHEN title LIKE 'Hộp%' THEN 2 ELSE 3 END"))

In [22]:
products_df.printSchema()

root
 |-- title: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- price: double (nullable = true)
 |-- sale_price: double (nullable = true)
 |-- category_id: integer (nullable = false)
 |-- description: string (nullable = true)
 |-- detail: string (nullable = true)
 |-- link_shopee: string (nullable = true)
 |-- link_zalo: string (nullable = true)
 |-- link_instagram: string (nullable = true)
 |-- meta_title: string (nullable = false)
 |-- meta_description: string (nullable = false)



In [23]:
products_df.show()

+--------------------+--------------------+--------+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------------+
|               title|                slug|   price|sale_price|category_id|         description|              detail|         link_shopee|           link_zalo|      link_instagram|meta_title|meta_description|
+--------------------+--------------------+--------+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------------+
|Bình giữ nhiệt ch...|binh-giu-nhiet-ch...|339286.0|  237500.0|          1|Bình giữ nhiệt Su...|- Dung tích lớn 5...|https://shopee.vn...|https://zalo.me/9...|https://www.insta...|          |                |
|Bình giữ nhiệt He...|binh-giu-nhiet-he...|565098.0|  395569.0|          1|Bình giữ nhiệt ca...|Bình giữ nhiệt ca...|https://shopee.vn...|https://zalo.me/9...|https

In [14]:
# products_df.write.option('header', 'true') \
#     .option('encoding', 'UTF-8') \
#     .mode('overwrite') \
#     .csv('E:/cloud_data/')

# DB properties

In [ ]:
DB_URL = 'jdbc:mysql://cloudproject.cp2a0mms4f86.ap-southeast-1.rds.amazonaws.com:3306/cloud'
DB_USER = 'admin'
DB_PASSWORD = '12345678'

# Write products table

In [24]:
jdbc_options = {
    "url": DB_URL,
    "user": DB_USER,
    "password": DB_PASSWORD,
    "dbtable": 'products',
    "driver": "com.mysql.cj.jdbc.Driver"
}


products_df.write.jdbc(
    url=DB_URL,
    table='products',
    mode='append',
    properties=jdbc_options
)

# Write product_options table

In [107]:
categorical_products = df_exploded \
    .withColumn('product_options', explode('entries_exploded.categorical_products')) \
    .select(col('entries_exploded.title').alias('product_title'), 'product_options.*') \
    .withColumn('price', col('price') * 1000)
    
categorical_products.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------+-----------------------+--------+----------+--------+-------+-------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|product_title                                                                                                           |title                  |price   |sale-price|currency|options|img                                                                      |sub-img                                                                      |
+------------------------------------------------------------------------------------------------------------------------+-----------------------+--------+----------+--------+-------+-------------------------------------------------------------------------+-----------------------------------------------------------------------

In [108]:
products_id_df = spark.read.jdbc(
        url=DB_URL,
        table='products',
        properties=jdbc_options
    ).select('id', 'title', 'price')

products_id_df.printSchema()

products_id_df.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- price: decimal(15,2) (nullable = true)

+---+------------------------------------------------------------------------------------------------------------------------+---------+
|id |title                                                                                                                   |price    |
+---+------------------------------------------------------------------------------------------------------------------------+---------+
|1  |Bình giữ nhiệt chủ đề Vincent Van Gogh cao cấp 520ml                                                                    |339286.00|
|2  |Bình giữ nhiệt Heoscu cao cấp inox 316 600ml                                                                            |565098.00|
|3  |Bình nước cho trẻ em 500ml                                                                                              |207660.00|
|4  |Bình giữ nhiệt bộ sưu tập Arles chủ đề Vincent van 

In [109]:
product_options_df = categorical_products.alias('c') \
    .join(products_id_df.alias('p'), categorical_products.product_title==products_id_df.title, 'inner') \
    .select(
        col('p.id').alias('product_id'),
        col('c.title').alias('option_name'),
        coalesce(expr('p.price - c.price'), lit(0)).alias('price_adjustment')
    )
    
product_options_df.show(truncate=False)

+----------+-----------------------+----------------+
|product_id|option_name            |price_adjustment|
+----------+-----------------------+----------------+
|1         |Salvia And Other Flower|0.0             |
|1         |The Van Gogh           |0.0             |
|1         |Sun Flower             |0.0             |
|1         |Salvia And Flower      |0.0             |
|1         |Green Wheat Field      |0.0             |
|2         |Pink                   |0.0             |
|2         |White                  |0.0             |
|2         |Green                  |0.0             |
|3         |Yellow                 |0.0             |
|3         |Pink                   |0.0             |
|3         |White                  |0.0             |
|3         |Green                  |0.0             |
|4         |Wheat Fields Auvers    |0.0             |
|4         |Langlois Bridge        |0.0             |
|4         |Garden In Montmartre   |0.0             |
|5         |Wheat Fields Cro

In [48]:
product_options_df.write.jdbc(
    url=DB_URL,
    table='product_options',
    mode='append',
    properties=jdbc_options
)

# Write product_images table

In [110]:
product_options_id_df = spark.read.jdbc(
    url=DB_URL,
    table='product_options',
    properties=jdbc_options
).select(col('id').alias('product_option_id'), 'product_id', 'option_name')

product_options_id_df.show(truncate=False)

+-----------------+----------+-----------------------+
|product_option_id|product_id|option_name            |
+-----------------+----------+-----------------------+
|1                |1         |Salvia And Other Flower|
|2                |1         |The Van Gogh           |
|3                |1         |Sun Flower             |
|4                |1         |Salvia And Flower      |
|5                |1         |Green Wheat Field      |
|6                |2         |Pink                   |
|7                |2         |White                  |
|8                |2         |Green                  |
|9                |3         |Yellow                 |
|10               |3         |Pink                   |
|11               |3         |White                  |
|12               |3         |Green                  |
|13               |4         |Wheat Fields Auvers    |
|14               |4         |Langlois Bridge        |
|15               |4         |Garden In Montmartre   |
|16       

In [112]:
product_image_df = products_id_df.alias('p') \
    .join(df_exploded.alias('e'), products_id_df.title==df_exploded.entries_exploded.title, 'inner') \
    .select(
        col('p.id').alias('product_id'), 
        'e.entries_exploded.img', 'e.entries_exploded.sub-img'
    )
    
product_image_df.show(truncate=False)

+----------+-----------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|img                                                        |sub-img                                                                                                                                                                                                                                                                                                                                               

In [113]:
product_sub_img_df = product_image_df \
    .select(
        'product_id',
        explode('sub-img').alias('image_url')
    ) \
    .withColumn('is_main', lit(0)) \
    .withColumn('display_order', expr('ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY image_url)')) 

product_sub_img_df.show(truncate=False)

product_image_df = product_image_df \
    .withColumn('is_main', lit(1)) \
    .withColumn('display_order', lit(0)) \
    .select(
        'product_id',
        col('img').alias('image_url'),
        'is_main',
        'display_order'
    )
    
product_image_df.show(truncate=False)

+----------+-------------------------------------------------------------+-------+-------------+
|product_id|image_url                                                    |is_main|display_order|
+----------+-------------------------------------------------------------+-------+-------------+
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet1.webp|0      |1            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet2.webp|0      |2            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet3.webp|0      |3            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet4.webp|0      |4            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet5.webp|0      |5            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet6.webp|0      |6            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet7.webp|0      |7            |
|1         |{{webRoot}}/assets

In [115]:
final_product_images = product_image_df \
    .union(product_sub_img_df) \
    .orderBy('product_id', desc('is_main'), 'display_order')

final_product_images.show(truncate=False)

+----------+-------------------------------------------------------------+-------+-------------+
|product_id|image_url                                                    |is_main|display_order|
+----------+-------------------------------------------------------------+-------+-------------+
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/5Binh.png    |1      |0            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet1.webp|0      |1            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet2.webp|0      |2            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet3.webp|0      |3            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet4.webp|0      |4            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet5.webp|0      |5            |
|1         |{{webRoot}}/assets/images/products/binhGiuNhiet/chitiet6.webp|0      |6            |
|1         |{{webRoot}}/assets

In [ ]:
final_product_images.write.jdbc(
    url=DB_URL,
    table='product_images',
    mode='append',
    properties=jdbc_options
)

# Write product_option_images table

In [118]:
product_options_image_df = categorical_products.alias('c') \
    .join(products_id_df.alias('p'), categorical_products.product_title==products_id_df.title, 'inner') \
    .join(product_options_id_df.alias('po'), 
          (categorical_products.title==product_options_id_df.option_name) & (products_id_df.id==product_options_id_df.product_id)) \
    .select(
        'product_id',
        'po.product_option_id',
        'c.img',
        'c.sub-img'
    )
    
product_options_image_df.orderBy('product_id', 'product_option_id').show(truncate=False)

+----------+-----------------+-------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|product_id|product_option_id|img                                                                      |sub-img                                                                      |
+----------+-----------------+-------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|1         |1                |                                                                         |[{{webRoot}}/assets/images/products/binhGiuNhiet/salviaAndOtherFlower/d1.png]|
|1         |2                |{{webRoot}}/assets/images/products/binhGiuNhiet/vanGogh/main.webp        |[{{webRoot}}/assets/images/products/binhGiuNhiet/vanGogh/d1.webp]            |
|1         |3                |{{webRoot}}/assets/images/products/binhGiuNhiet/sunFlow

In [119]:
product_options_sub_img_df = product_options_image_df \
    .select(
        'product_option_id',
        explode('sub-img').alias('image_url')
    ) \
    .withColumn('is_main', lit(0)) \
    .withColumn('display_order', expr('ROW_NUMBER() OVER (PARTITION BY product_option_id ORDER BY image_url)')) 

# product_options_sub_img_df.show(truncate=False)

product_options_image_df = product_options_image_df \
    .withColumn('is_main', lit(1)) \
    .withColumn('display_order', lit(0)) \
    .select(
        'product_option_id',
        col('img').alias('image_url'),
        'is_main',
        'display_order'
    )
    
# product_options_image_df.show(truncate=False)

In [120]:
final_product_option_images = product_options_image_df \
    .union(product_options_sub_img_df) \
    .orderBy('product_option_id', desc('is_main'), 'display_order')

final_product_option_images.show(truncate=False)

+-----------------+---------------------------------------------------------------------------+-------+-------------+
|product_option_id|image_url                                                                  |is_main|display_order|
+-----------------+---------------------------------------------------------------------------+-------+-------------+
|1                |                                                                           |1      |0            |
|1                |{{webRoot}}/assets/images/products/binhGiuNhiet/salviaAndOtherFlower/d1.png|0      |1            |
|2                |{{webRoot}}/assets/images/products/binhGiuNhiet/vanGogh/main.webp          |1      |0            |
|2                |{{webRoot}}/assets/images/products/binhGiuNhiet/vanGogh/d1.webp            |0      |1            |
|3                |{{webRoot}}/assets/images/products/binhGiuNhiet/sunFlower/main.webp        |1      |0            |
|3                |{{webRoot}}/assets/images/products/bi

In [122]:
final_product_option_images.write.jdbc(
    url=DB_URL,
    table='product_option_images',
    mode='append',
    properties=jdbc_options
)

In [100]:
spark.stop()